# Deleting the empty folders

Before preprocessing the data and training a model, there are many directories or folder which do not contain 'part files'. It is easier to delete these folders since they are not significant.

In [1]:
import os
import shutil
import numpy

from pyspark.ml import Pipeline
        
from pyspark.ml.feature import Tokenizer, RegexTokenizer, StopWordsRemover, HashingTF, IDF, OneHotEncoder, StringIndexer, VectorAssembler

from pyspark.sql.types import IntegerType, StringType, ArrayType
from pyspark.streaming import StreamingContext
from pyspark.sql import Row
from pyspark.sql.functions import udf, struct, array, col, lit, monotonically_increasing_id

from pyspark.sql.types import StructType
from pyspark.sql.types import StructField

from pyspark.ml.classification import NaiveBayes, NaiveBayesModel
from pyspark.ml.linalg import Vectors
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.evaluation import MultilabelMetrics
from pyspark.mllib.util import MLUtils



In [2]:
## Determine directories of the data and the location for the cleaned data
# directory = r'/Users/Simon/Documents/GitHub/adana_task3/Spark_Temp'
# new_directory = r'/Users/Simon/Documents/GitHub/adana_task3/Spark_Cleaned'

directory = r'/Users/Simon/Documents/GitHub/adana_task3/Spark_Data'
new_directory = r'/Users/Simon/Documents/GitHub/adana_task3/AllData2'

# directory = r'/Users/Simon/Desktop/DataStream2Cedric'
# new_directory = r'/Users/Simon/Desktop/AllData1'

filenames = os.listdir(directory) #a list of all the folder names

shutil.rmtree(new_directory, ignore_errors=False, onerror=None) #Reset (delete) the new directory
shutil.copytree(directory, new_directory) #Copy the data to a new directory to built in some redundancy 


'/Users/Simon/Documents/GitHub/adana_task3/AllData2'

In [3]:
## Removing the folders which do not contain 'part files'. These folders do not contain any useful data.
counter = 0
removed_folders = []
for i in range(len(filenames)):
    remove = False
    folder = new_directory + "/" + filenames[i]
    
    if filenames[i] == '.DS_Store' or filenames[i] == 'Icon\r':
        continue
        
    if len(os.listdir(folder)) <= 1:
        remove = True
        removed_folders.append(filenames[i])
        counter += 1
        shutil.rmtree(folder, ignore_errors=False, onerror=None)
        
print("The Total Number of Original Folders is: ", len(filenames))
print()

print("The Total Number of Removed Folders is: ", counter)
print()

print("Remaining folders: ", (len(filenames)) - counter )
# print(removed_folders)


The Total Number of Original Folders is:  4296

The Total Number of Removed Folders is:  4274

Remaining folders:  22


In [4]:
def load_rdd(base_directory):
    # Get all the directory names of the saved myoutput folders
    foldernames = os.listdir(base_directory)
    
    # Create list of full directories
    full_directories = []
    
    for i in range(len(foldernames)):
        
        if foldernames[i] == '.DS_Store':
            continue
        
        directory_temp = base_directory + "/" + foldernames[i]
        full_directories.append(directory_temp)
    
    print("Number of directories included: ", len(full_directories))
    
    df = spark.read.format('json').load(path=full_directories)

    return df

# base_directory = r'/Users/Simon/Documents/GitHub/adana_task3/Spark_Cleaned/myoutput-1586797640000/part-00003'
# base_directory = r'/Users/Simon/Documents/GitHub/adana_task3/Spark_Cleaned/myoutput-1586797640000'
# base_directory = r'/Users/Simon/Documents/GitHub/adana_task3/Spark_Data_Limited'
# base_directory = r'/Users/Simon/Documents/GitHub/adana_task3/Spark_Cleaned'

# df = load_rdd(base_directory) 
# print("Number of part files included: ", len(df.toPandas()))

# df.write.csv(r'/Users/Simon/Documents/GitHub/adana_task3/All_Data.csv')



The idea is to filter our data so that approximatily the same number of instances of each class are present in the data!